# Hello SYCL and DPC++

***
In this xpublog, we meet Nermal the kitten and his favorite book (<a href="https://link.springer.com/book/10.1007%2F978-1-4842-5574-2" target="_blank">Data Parallel C++</a>).  We use a SYCL kernel to lighten him up (his image is a little tensor).

Completing this notebook counts toward our daily goals for viewing some cute kitten images/videos on the internet.

***

##  New to Jupyter?

For the most part... just click the run icon (▶) in the little panel of buttons at the top of this browser pane. The panel looks something like this (where play is the sixth item):<br/><img src="Assets/jupyter.buttons.png">
    
Click the ▶ button over and over... and soon you will have compiled, run, and viewed the output of this SYCL program compiled by the DPC++ compiler.  Now that you know about the little panel of buttons, you should be unstoppable.
    

***

##  Editing the hello.cpp code
The Jupyter cell below with the gray background can be edited in-place and saved.

The first line of the cell contains the command **%%writefile 'hello.cpp'** This tells the input cell to save the contents of the cell into a file named 'simple.cpp' in your current directory (usually your home directory). As we edit the cell and run it in the Jupyter notebook, it will save our changes into that file.

The code below is some DPC++ code to get us started in the DevCloud environment. We can simply inspect the code - it is ready to run, no modifications necessary. Run the first cell to create the file, then run the cell below it to compile and execute the code.
1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/hello.cpp
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <opencv2/core.hpp>
#include <opencv2/imgcodecs.hpp>
#include <opencv2/highgui.hpp>
#include <iostream>
#include <algorithm>
#include <CL/sycl.hpp>

using namespace cv;
using namespace std;
using namespace sycl;

int main(){
  string image_path = "NermalWithBook540.jpg";
    
  //# define queue which has default device associated for offload
  queue q;
  // for this exercise, we did not control what type of device we are offloading to,
  // so this output will tell us what we were assigned
  std::cout << "Device: " << q.get_device().get_info<info::device::name>() << std::endl;

  Mat img = imread(image_path, IMREAD_COLOR);
  if(img.empty()) {
    std::cout << "Could not read the image: " << image_path << std::endl;
    return 1;
  }

#if 0
  // the SYCL kernel, after the endif, will do the same as this C++ code would do
  for( int y = 0; y < img.rows; y++ ) {
    for( int x = 0; x < img.cols; x++ ) {
      for( int c = 0; c < img.channels(); c++ ) {
        img.at<Vec3b>(y,x)[c] =
          clamp( img.at<Vec3b>(y,x)[c] + 50, 0, 255 );
      }
    }
  }
#endif
  {
    CV_Assert(img.isContinuous());
    CV_CheckTypeEQ(img.type(), CV_8UC3, "");
    buffer<uint8_t, 3> frame_buffer(img.data, range<3>(img.rows, img.cols, 3));

    q.submit([&](handler& cgh) {
      auto pixels = frame_buffer.get_access<access::mode::read_write>(cgh);
      cgh.parallel_for(range<3>(img.rows, img.cols, 3), [=](item<3> item) {
        uint8_t p = pixels[item];
        pixels[item] = sycl::clamp(p+50,0,255);
      });
    });
    q.wait_and_throw();
  }

  imwrite("EnlightenedNermalWithBook.png", img);

  return 0;
}

***

### Build and Run
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 run_hello.sh;if [ -x "$(command -v qsub)" ]; then ./q run_hello.sh; else ./run_hello.sh; fi

_If the Jupyter cells are not responsive or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_

***

Wait for the above to be DONE before proceeding... or else the output image will not be ready to view!

## Hello Nermal (a Kitten with a Book)
The orginal image looks like this:

<img src="Assets/NermalWithBook540.jpg">

The output image looks like this (select this cell and click Run ▶ to refresh the image):

<img src="EnlightenedNermalWithBook.png">

***

## Additional resources realted to this xpublog

* [Link back to the main text and video for this xpublog](https://software.intel.com/content/www/us/en/develop/articles/xpublog0001-hello-sycl-and-dpcpp.html "xpublog 1+2+3 - Hello SYCL")
* [Link to github where this code is kept](https://github.com/jamesreinders/xpublog "github repository")

#### Training for Intel® oneAPI
* [More extensive DevCloud+Jupyter for oneAPI](https://devcloud.intel.com/oneapi/get_started/baseTrainingModules/ "oneAPI basic training in Jupyter")
* [Beyond the basics - training for oneAPI](https://software.intel.com/content/www/us/en/develop/tools/oneapi/training.html "oneAPI training")

#### Installing Intel® oneAPI on your own systems
* [Get Started with oneAPI for Linux*](https://software.intel.com/en-us/get-started-with-intel-oneapi-linux "Install / your / own!")
* [Get Started with oneAPI for Windows*](https://software.intel.com/en-us/get-started-with-intel-oneapi-windows "Install \ your \ own!")

#### Open books, magazine, and standards
* [Data Parallel C++ / SYCL Book](https://link.springer.com/book/10.1007%2F978-1-4842-5574-2 "download the (free) book, do not wait for the movie")
* [Parallel Universe - long standing journal of interesting articles](https://software.intel.com/content/www/us/en/develop/tools/parallel-universe-magazine.html "Parallel or Perish!")
* [oneAPI Specification elements](https://www.oneapi.com/spec/ "oneAPI specification for multivendor open support for XPUs")
* [SYCL Specification](https://www.khronos.org/registry/SYCL/ "SYCL standard")
* [OpenCV](https://docs.opencv.org/master/index.html "OpenCV")
* [openVINO](https://docs.openvinotoolkit.org/latest/index.html "OpenVINO")


***
## Reset Notebook

If you are experiencing any issues with your notebook, or if you just want to start fresh, run the below cell.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/xpublog/0003_Hello_SYCL/ ~/xpublog/0003_Hello_SYCL
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

***